生成器因为可以终止运行保存状态,所以可以实现协程

In [1]:
from __future__ import print_function

In [3]:
def consumer():
    while True:
        line = yield
        print(line.upper())
        
def producter():
    with open("README.md") as f:
        for i,line in enumerate(f):
            yield line
            print("processed line {}".format(i))
c = consumer()
try :
    c.next()
except:
    next(c)

for line in producter():
    c.send(line)

# 函数式编程

processed line 0


processed line 1
PYTHON是一门相对比较潮的语言,也是一门相对学院派的语言(要不也不会在大学里特别受欢迎了).为啥这么说呢,因为现在流行的函数式编程特性它老早就有了.不过和正儿八经的函数式编程语言如LISP呀HASKELL呀SCALA呀这种相比,PYTHON的函数式编程还只是半吊子罢了.但多数情况下PYTHON的那些函数式编程特性已经很够用而且可以帮助用户节省大量代码了."人生苦短,我用PYTHON"很多时候也是源于它的那些函数式编程特性.

processed line 2


processed line 3
这部分的主要内容有:

processed line 4


processed line 5
+ 匿名函数和函数变量

processed line 6
+ 高阶函数

processed line 7
+ 表达式和解析

processed line 8
+ 生成器和生成器表达式

processed line 9
+ 珂里化

processed line 10
+ 作用域

processed line 11
+ 闭包

processed line 12


在 2.7版本中生成器并没有完整的携程支持,比如一个协程依赖另一个协程,且需要由最外层向内层进行传值,就没办法实现.
在3.4中终于有了新的关键字`yield from...`来实现这个功能了
> 一个异步二叉树的例子

基本版:

In [11]:
from asyncio import coroutine

class Node(object):
    def __init__(self,value):
        self.left = []
        self.value = value
        self.right = []
        
    def iterate(self):
        for node in self.left:
            yield node.value
        yield self.value
        for node in self.right:
            yield node.value
def main():
    root = Node(0)
    root.left = [Node(i) for i in [1,2,3]]
    root.right = [Node(i) for i in [4,5,6]]
    for value in root.iterate():
        print(value)
        
main()

1
2
3
0
4
5
6


递归版:

In [12]:
from asyncio import coroutine

class Node(object):
    def __init__(self,value):
        self.left = []
        self.value = value
        self.right = []
        
    def iterate(self):
        for node in self.left:
            for value in node.iterate():
                yield value
        yield self.value
        for node in self.right:
            for value in node.iterate():
                yield value
def main():
    root = Node(0)
    root.left = [Node(i) for i in [1,2,3]]
    root.right = [Node(i) for i in [4,5,6]]
    for value in root.iterate():
        print(value)
        
main()

1
2
3
0
4
5
6


yield from 递归版

In [13]:
from asyncio import coroutine

class Node(object):
    def __init__(self,value):
        self.left = []
        self.value = value
        self.right = []
        
    def iterate(self):
        for node in self.left:
            yield from node.iterate()
        yield self.value
        for node in self.right:
            yield from node.iterate()
def main():
    root = Node(0)
    root.left = [Node(i) for i in [1,2,3]]
    root.right = [Node(i) for i in [4,5,6]]
    for value in root.iterate():
        print(value)
        
main()

1
2
3
0
4
5
6


## asyncio模型

In [14]:
import asyncio

@asyncio.coroutine
def hello():
    print("Hello world!")
    # 异步调用asyncio.sleep(1):
    r = yield from asyncio.sleep(1)
    print("Hello again!")

# 获取EventLoop:
loop = asyncio.get_event_loop()
# 执行coroutine
loop.run_until_complete(hello())
loop.close()

Hello world!
Hello again!


@asyncio.coroutine把一个generator标记为coroutine类型，然后，我们就把这个coroutine扔到EventLoop中执行。

hello()会首先打印出Hello world!，然后，yield from语法可以让我们方便地调用另一个generator。由于asyncio.sleep()也是一个coroutine，所以线程不会等待asyncio.sleep()，而是直接中断并执行下一个消息循环。当asyncio.sleep()返回时，线程就可以从yield from拿到返回值（此处是None），然后接着执行下一行语句。

把asyncio.sleep(1)看成是一个耗时1秒的IO操作，在此期间，主线程并未等待，而是去执行EventLoop中其他可以执行的coroutine了，因此可以实现并发执行。

我们用Task封装两个coroutine试试：

In [1]:
import threading
import asyncio

@asyncio.coroutine
def hello():
    print('Hello world! (%s)' % threading.currentThread())
    yield from asyncio.sleep(1)
    print('Hello again! (%s)' % threading.currentThread())

loop = asyncio.get_event_loop()
tasks = [hello(), hello()]
loop.run_until_complete(asyncio.wait(tasks))
loop.close()

Hello world! (<_MainThread(MainThread, started 140735127961600)>)
Hello world! (<_MainThread(MainThread, started 140735127961600)>)
Hello again! (<_MainThread(MainThread, started 140735127961600)>)
Hello again! (<_MainThread(MainThread, started 140735127961600)>)


由打印的当前线程名称可以看出，两个coroutine是由同一个线程并发执行的。

如果把asyncio.sleep()换成真正的IO操作，则多个coroutine就可以由一个线程并发执行。

我们用asyncio的异步网络连接来获取sina、sohu和163的网站首页：

In [1]:
import asyncio

@asyncio.coroutine
def wget(host):
    print('wget %s...' % host)
    connect = asyncio.open_connection(host, 80)
    reader, writer = yield from connect
    header = 'GET / HTTP/1.0\r\nHost: %s\r\n\r\n' % host
    writer.write(header.encode('utf-8'))
    yield from writer.drain()
    while True:
        line = yield from reader.readline()
        if line == b'\r\n':
            break
        print('%s header > %s' % (host, line.decode('utf-8').rstrip()))
    # Ignore the body, close the socket
    writer.close()

loop = asyncio.get_event_loop()
tasks = [wget(host) for host in ['www.sina.com.cn', 'www.sohu.com', 'www.163.com']]
loop.run_until_complete(asyncio.wait(tasks))
loop.close()

wget www.sina.com.cn...
wget www.163.com...
wget www.sohu.com...
www.sina.com.cn header > HTTP/1.1 200 OK
www.sina.com.cn header > Content-Type: text/html
www.sina.com.cn header > Vary: Accept-Encoding
www.sina.com.cn header > X-Powered-By: schi_v1.02
www.sina.com.cn header > Server: nginx
www.sina.com.cn header > Date: Sun, 28 Feb 2016 05:35:08 GMT
www.sina.com.cn header > Last-Modified: Sun, 28 Feb 2016 05:33:16 GMT
www.sina.com.cn header > Expires: Sun, 28 Feb 2016 05:36:08 GMT
www.sina.com.cn header > Cache-Control: max-age=60
www.sina.com.cn header > Age: 40
www.sina.com.cn header > Content-Length: 527053
www.sina.com.cn header > X-Cache: HIT from localhost
www.sina.com.cn header > Connection: close
www.163.com header > HTTP/1.0 200 OK
www.163.com header > Expires: Sun, 28 Feb 2016 05:37:09 GMT
www.163.com header > Date: Sun, 28 Feb 2016 05:35:49 GMT
www.163.com header > Server: nginx
www.163.com header > Content-Type: text/html; charset=GBK
www.163.com header > Vary: Accept-Encod

用asyncio提供的@asyncio.coroutine可以把一个generator标记为coroutine类型，然后在coroutine内部用yield from调用另一个coroutine实现异步操作。

为了简化并更好地标识异步IO，从Python 3.5开始引入了新的语法async和await，可以让coroutine的代码更简洁易读。

请注意，async和await是针对coroutine的新语法，要使用新的语法，只需要做两步简单的替换：

+ 把`@asyncio.coroutine`替换为`async`；
+ 把`yield from`替换为`await`。


In [1]:
import threading
import asyncio

async def hello():
    print("Hello world!")
    r = await asyncio.sleep(1)
    print("Hello again!")

In [2]:
loop = asyncio.get_event_loop()
tasks = [hello(), hello()]
loop.run_until_complete(asyncio.wait(tasks))
loop.close()

Hello world!
Hello world!
Hello again!
Hello again!
